In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

# 定义一个简单的模型
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(10, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 创建并初始化模型
model = SimpleModel()

# 模拟训练过程，随机初始化权重
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 假设模型已经训练过了，这里保存其状态字典
torch.save({
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict(),
}, 'original_model.pth')


print("origin_static: ")
for key,value in model.state_dict().items():
    print(key, " : ", value.shape)


origin_static: 
fc1.weight  :  torch.Size([20, 10])
fc1.bias  :  torch.Size([20])
fc2.weight  :  torch.Size([10, 20])
fc2.bias  :  torch.Size([10])
fc3.weight  :  torch.Size([1, 10])
fc3.bias  :  torch.Size([1])


In [7]:

# 模拟剪枝操作：删除 `fc2` 层的权重（假设剪枝）
pruned_model = SimpleModel()
pruned_state_dict = torch.load('original_model.pth')['state_dict']
# 剪掉不需要的层（这里删除 `fc2` 层）
pruned_state_dict.pop('fc2.weight')
pruned_state_dict.pop('fc2.bias')

# 保存剪枝后的权重
torch.save({
    'state_dict': pruned_state_dict,
}, 'pruned_bafpp.pth')

print("pruned_static: ")
for key,value in pruned_state_dict.items():
    print(key, " : ", value.shape)

print("Pruned model saved as 'pruned_bafpp.pth'.")


pruned_static: 
fc1.weight  :  torch.Size([20, 10])
fc1.bias  :  torch.Size([20])
fc3.weight  :  torch.Size([1, 10])
fc3.bias  :  torch.Size([1])
Pruned model saved as 'pruned_bafpp.pth'.


SimpleModel(
  (fc1): Linear(in_features=10, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=1, bias=True)
)


In [ ]:
# 重新定义一个模型，加载剪枝后的权重
new_model = SimpleModel()

# 加载剪枝后的权重
checkpoint = torch.load('pruned_bafpp.pth')
pruned_state_dict = checkpoint['state_dict']

# 手动调整参数名称或去掉不必要的层
pruned_state_dict = {k: v for k, v in pruned_state_dict.items() if k in new_model.state_dict()}

# 更新模型
new_model.load_state_dict(pruned_state_dict, strict=False)

# 打印模型结构，查看哪些层被加载
print(new_model)


In [ ]:
checkpoint = torch.load('pruned_bafpp.pth')
pruned_state_dict = checkpoint['state_dict']

# 手动调整参数名称或去掉不必要的层
pruned_state_dict = {k: v for k, v in pruned_state_dict.items() if k in model.state_dict()}

# 更新模型
model.load_state_dict(pruned_state_dict, strict=False)


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import copy

# 定义原始模型
class OriginalModel(nn.Module):
    def __init__(self):
        super(OriginalModel, self).__init__()
        self.fc1 = nn.Linear(10, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 定义剪枝后的模型（例如，减少fc2的输入特征数从20到15）
class PrunedModel(nn.Module):
    def __init__(self, pruned_fc1_out_features=15, pruned_fc2_in_features=15):
        super(PrunedModel, self).__init__()
        self.fc1 = nn.Linear(10, pruned_fc1_out_features)
        self.fc2 = nn.Linear(pruned_fc2_in_features, 10)
        self.fc3 = nn.Linear(10, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def print_model_structure(model, title):
    print(f"\n{title}")
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            print(f"{name}: {module}")

def main():
    # 1. 初始化并保存原始模型
    original_model = OriginalModel()
    print_model_structure(original_model, "原始模型结构:")
    
    # 模拟训练过程（这里只是随机初始化权重）
    optimizer = optim.SGD(original_model.parameters(), lr=0.01)
    
    # 保存原始模型的权重
    torch.save({
        'state_dict': original_model.state_dict(),
        'optimizer': optimizer.state_dict(),
    }, 'original_model.pth')
    print("\n原始模型已保存为 'original_model.pth'.")

    # 2. 定义剪枝后的模型（例如，减少fc1的输出特征数从20到15）
    pruned_fc1_out_features = 15
    pruned_fc2_in_features = pruned_fc1_out_features  # fc2的输入特征数应与fc1的输出特征数一致
    pruned_model = PrunedModel(pruned_fc1_out_features=pruned_fc1_out_features,
                               pruned_fc2_in_features=pruned_fc2_in_features)
    print_model_structure(pruned_model, "剪枝后的模型结构:")
    
    # 3. 复制并适配权重
    original_state_dict = torch.load('original_model.pth')['state_dict']
    pruned_state_dict = pruned_model.state_dict()

    # 复制fc1的权重和偏置（保留前15个输出特征）
    pruned_state_dict['fc1.weight'] = original_state_dict['fc1.weight'][:pruned_fc1_out_features, :]
    pruned_state_dict['fc1.bias'] = original_state_dict['fc1.bias'][:pruned_fc1_out_features]

    # 复制fc2的权重和偏置（保留前15个输入特征）
    pruned_state_dict['fc2.weight'] = original_state_dict['fc2.weight'][:, :pruned_fc2_in_features]
    pruned_state_dict['fc2.bias'] = original_state_dict['fc2.bias']

    # 复制fc3的权重和偏置（保持不变）
    pruned_state_dict['fc3.weight'] = original_state_dict['fc3.weight']
    pruned_state_dict['fc3.bias'] = original_state_dict['fc3.bias']

    # 4. 加载适配后的权重到剪枝后的模型
    pruned_model.load_state_dict(pruned_state_dict)
    print("\n剪枝后的模型权重已加载.")

    # 5. 保存剪枝后的模型
    torch.save({
        'state_dict': pruned_model.state_dict(),
    }, 'pruned_model.pth')
    print("剪枝后的模型已保存为 'pruned_model.pth'.")

    # 6. 加载剪枝后的模型到新模型实例中
    loaded_pruned_model = PrunedModel(pruned_fc1_out_features=pruned_fc1_out_features,
                                     pruned_fc2_in_features=pruned_fc2_in_features)
    checkpoint = torch.load('pruned_model.pth')
    loaded_pruned_model.load_state_dict(checkpoint['state_dict'])
    print_model_structure(loaded_pruned_model, "加载剪枝后模型的结构:")

    # 7. 比较原始模型和剪枝后模型的结构差异
    print("\n比较原始模型和剪枝后的模型结构差异:")
    print(f"原始模型的 fc1 输出特征数: {original_model.fc1.out_features}")
    print(f"剪枝后模型的 fc1 输出特征数: {pruned_model.fc1.out_features}")
    print(f"加载剪枝后模型的 fc1 输出特征数: {loaded_pruned_model.fc1.out_features}")

if __name__ == "__main__":
    main()



原始模型结构:
fc1: Linear(in_features=10, out_features=20, bias=True)
fc2: Linear(in_features=20, out_features=10, bias=True)
fc3: Linear(in_features=10, out_features=1, bias=True)

原始模型已保存为 'original_model.pth'.

剪枝后的模型结构:
fc1: Linear(in_features=10, out_features=15, bias=True)
fc2: Linear(in_features=15, out_features=10, bias=True)
fc3: Linear(in_features=10, out_features=1, bias=True)

剪枝后的模型权重已加载.
剪枝后的模型已保存为 'pruned_model.pth'.

加载剪枝后模型的结构:
fc1: Linear(in_features=10, out_features=15, bias=True)
fc2: Linear(in_features=15, out_features=10, bias=True)
fc3: Linear(in_features=10, out_features=1, bias=True)

比较原始模型和剪枝后的模型结构差异:
原始模型的 fc1 输出特征数: 20
剪枝后模型的 fc1 输出特征数: 15
加载剪枝后模型的 fc1 输出特征数: 15
